## Test problem for GWE

This worked example sets up a MODFLOW 6 simulation with 2 GWF models and 2 GWE models and explores the sensitivity of the Brooks-Corey epsilon parameter within the UZF package and what impact it can have on the flow solution

In [ ]:
# Imports
import os
import sys

import flopy
import pandas as pd
import numpy as np
import pytest
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.collections import LineCollection
import matplotlib
from pathlib import Path
from IPython.display import HTML

### Set simulation name and workspace

In [ ]:
sim_name = "unsat-comp"
sim_ws = Path("../data/gwe-uze")

### Set some model input values

In [ ]:
# Model units
length_units = "meters"
time_units = "seconds"

# Solver parameters
nouter, ninner = 100, 300
hclose, rclose, relax = 1e-8, 1e-8, 0.97

# Set some parameter values
nper = 365  # Number of periods
nstp = 1  # Number of time steps
perlen = 86400  # Simulation time length ($s$)
nlay = 120  # Number of layers
nrow = 1  # Number of rows
ncol = 1  # Number of columns
system_length = 60.0  # Length of system ($m$)
delr = 1.0  # Column width ($m$)
delc = 1.0  # Row width ($m$)
delv_str = "ranges from 0.1 to 1"  # Layer thickness
top = 60.0  # Top of the model ($m$)
keff = 3e-6
hydraulic_conductivity = keff  # Hydraulic conductivity ($m s^{-1}$)
alphal = 0.0  # Longitudinal dispersivity ($m$)
alphat = 0.0  # Transverse dispersivity ($m$)
T_az = 5  # Ambient temperature ($^o C$)
dT = 5  # Temperature variation ($^o C$)
cpw = 4174.0
cps = 1600.0
rhow = 1000.0
rhos = 2630.0
lambda_w = 0.58
lambda_s = 2
bulk_dens = rhos  # Bulk density ($kg/m^3$)
kd = cps/(cpw*rhow)  # Distribution coefficient (unitless)
ktw = lambda_w
lhv = 0
Zw = 55
Sw = 1
u_gradient = 0.3 / 60
darcy_flux = keff * (u_gradient)
lambda_s1 = 2.4
lambda_s2 = lambda_s1
thtr = 0.01
n1 = 0.24
n2 = n1

In [ ]:
# Stress period input
per_data = []
for k in range(nper):
    per_data.append((perlen, nstp, 1.0))
per_mf6 = per_data

# Geometry input
tp = top
botm = []
for i in range(nlay):
    if i == 0:
        botm.append(59.9)
    elif i == 119:
        botm.append(0.0)
    else:
        botm.append(60-i*0.5)


### Setting some boundary package data

In [ ]:
# Head input
chd_data = {}
chd_data[0] = [[(119, 0, 0), 53.875]]
    
chd_mf6 = chd_data

# Recharge input
wel_data = {}
wel_data[0] = [[(0, 0, 0), "prcp"]]
wel_mf6 = wel_data

# Initial temperature input
strt_conc = T_az* np.ones((nlay, 1, 1), dtype=np.float32)

### Read in time series input and prepare it for MODFLOW 6 time series utility

In [ ]:
# Boundary temperature input
data_path = "../data/gwe-uze"
df = pd.read_csv(os.path.join(data_path, 'Daily_dat.csv'), index_col=0)
df_2 = pd.read_csv(os.path.join(data_path, 'Daily_dat_2.csv'), index_col=0)
ctp_spd = {}
for i in range(nper):
    ctp_spd[i] = [[(0, 0, 0), df['TAVG_rmFrz'][i]]]  # ctp_spd.update({i: [[(0, 0, 0), df['TAVG_rmFrz'][i]]]})  

prcp_ts_data = []
temp_ts_data = []

for n in range(0, len(df)):
    time = n * 86400
    prcp = df['PRCP'][n] / 1000 / 86400
    temp = df['TAVG_rmFrz'][n]
    prcp_ts_data.append((time, prcp))
    temp_ts_data.append((time, temp))

prcp_ts_dict = {
    "filename": "prcp.ts",
    "time_series_namerecord": "prcp",
    "timeseries": prcp_ts_data,
    "interpolation_methodrecord": "linearend",
}

temp_ts_dict = {
    "filename": "temp.ts",
    "time_series_namerecord": "temp",
    "timeseries": temp_ts_data,
    "interpolation_methodrecord": "linearend",
}

prcp_ts_data_2 = []
temp_ts_data_2 = []
for n in range(0, len(df_2)):
    time = n * 86400
    prcp_2 = df_2['PRCP'][n] / 1000 / 86400
    temp_2 = df_2['TAVG_rmFrz'][n]
    prcp_ts_data_2.append((time, prcp_2))
    temp_ts_data_2.append((time, temp_2))

prcp_ts_dict_2 = {
    "filename": "prcp_2.ts",
    "time_series_namerecord": "prcp",
    "timeseries": prcp_ts_data_2,
    "interpolation_methodrecord": "linearend",
}

temp_ts_dict_2 = {
    "filename": "temp_2.ts",
    "time_series_namerecord": "temp",
    "timeseries": temp_ts_data_2,
    "interpolation_methodrecord": "linearend",
}


### Setup objects for UZF 

Includes two functions to help create the UZF input later on

In [ ]:
# transient uzf info
finf = "prcp"
extdp = 0.0
pet = 0.0
extwc = 0.0
zero = 0.0
uzf_spd = {
    0: [[0, finf, pet, extdp, extwc, zero, zero, zero]],
}

def add_uzf_obs(uz_obs_loc, obsnm, iuz, elev):
    uz_obs_loc.append((obsnm, "water-content", iuz, elev))
    return uz_obs_loc


def add_uzf_wc_profile_obs():
    uz_obs_loc = []
    iuz = 1
    depth = round((top-botm[0]) / 2, 2)
    elev = top - depth
    # Top uzf object (has a unique thickness)
    obsnm = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev)
    uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm, iuz, depth)

    # Loop through upper 10 cells
    increment = 0.1
    for iuz in np.arange(1, 12, 1):
        depth1 = depth  # 0.05
        depth2 = round(depth1 + increment, 2)  # 0.15
        depth3 = round(depth2 + increment, 2)  # 0.25
        depth4 = round(depth3 + increment, 2)  # 0.35
        depth5 = round(depth4 + increment, 2)  # 0.45
        elev1 = round(botm[iuz - 1] - depth1, 2)
        elev2 = round(botm[iuz - 1] - depth2, 2)
        elev3 = round(botm[iuz - 1] - depth3, 2)
        elev4 = round(botm[iuz - 1] - depth4, 2)
        elev5 = round(botm[iuz - 1] - depth5, 2)
        obsnm1 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev1)
        obsnm2 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev2)
        obsnm3 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev3)
        obsnm4 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev4)
        obsnm5 = "uzf" + str(iuz).zfill(2) + "_elev=" + str(elev5)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm1, iuz + 1, depth1)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm2, iuz + 1, depth2)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm3, iuz + 1, depth3)
        uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm4, iuz + 1, depth4)
        if iuz > 1:
            uz_obs_loc = add_uzf_obs(uz_obs_loc, obsnm5, iuz + 1, depth5)

    return uz_obs_loc


### Function for creating a GWF model

Since multiple GWF models will be added to the MODFLOW 6 simulation, define a function to do the work multiple times.

**Note**: The function call accepts an argument that defines the Brooks-Corey epsilon ($\epsilon$), the idea being that in this notebook, two calls to the this function would each instantiate a GWF model with a different value for $\epsilon$

In [ ]:
def add_gwf_model(sim, gwfname, eps=4):

    msg0 = 'EPSILON must be between 3.5 and 14.0'
    assert eps > 3.5 and eps < 15.0, msg0

    # Instantiating MODFLOW 6 groundwater flow model
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=gwfname,
        save_flows=True,
        newtonoptions = True,
        model_nam_file="{}.nam".format(gwfname),
    )

    # Instantiating MODFLOW 6 solver for flow model
    imsgwf = flopy.mf6.ModflowIms(
        sim,
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename="{}.ims".format(gwfname),
    )
    sim.register_ims_package(imsgwf, [gwfname])

    # Instantiating MODFLOW 6 discretization package
    flopy.mf6.ModflowGwfdis(
        gwf,
        length_units=length_units,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        filename="{}.dis".format(gwfname),
    )

    # Instantiating MODFLOW 6 node-property flow package
    flopy.mf6.ModflowGwfnpf(
        gwf,
        save_specific_discharge=True,
        icelltype=1,
        k=hydraulic_conductivity,
        filename="{}.npf".format(gwfname),
    )

    # Instantiate MODFLOW 6 storage package
    flopy.mf6.ModflowGwfsto(
        gwf,
        ss=1E-6,
        sy=n1,
        iconvert=1,
        #steady_state={0: True},
        filename="{}.sto".format(gwfname),
    )

    # Instantiating MODFLOW 6 initial conditions package for flow model
    flopy.mf6.ModflowGwfic(
        gwf, strt=Zw, filename="{}.ic".format(gwfname)
    )

    # Instantiating MODFLOW 6 constant head package
    flopy.mf6.ModflowGwfchd(
        gwf,
        stress_period_data=chd_mf6,
        filename="{}.chd".format(gwfname),
    )

    # Unsaturated-zone flow package
    uz_obs_loc = add_uzf_wc_profile_obs()
    uzf_obs = {
        f"{gwfname}.uzfobs": uz_obs_loc
    }
    if "scen" in gwfname:
        prcp_ts_dict_entry = prcp_ts_dict_2
    else:
        prcp_ts_dict_entry = prcp_ts_dict

    # iuzno  cellid landflg ivertcn surfdp vks thtr thts thti eps [bndnm]
    uzf_pkdat = [[0, (0, 0, 0), 1, 1, 0.00001, keff, thtr, thtr + n1, 0.025, eps]]

    # Continue building the UZF list of objects
    for iuzno in np.arange(1, nlay, 1):
        if iuzno < nlay - 1:
            ivertconn = iuzno + 1
        else:
            ivertconn = -1
        # Add the UZF object to the packagedata list object
        uzf_pkdat.append(
            [iuzno, (iuzno, 0, 0), 0, ivertconn, 0.0, keff, thtr, thtr + n1, 0.025, eps]
        )

    flopy.mf6.ModflowGwfuzf(
        gwf,
        print_flows=True,
        save_flows=True,
        wc_filerecord=gwfname + ".uzfwc.bin",
        timeseries=prcp_ts_dict_entry,
        simulate_et=False,
        simulate_gwseep=False,
        linear_gwet=False,
        boundnames=False,
        observations=uzf_obs,
        ntrailwaves=15,
        nwavesets=40,
        nuzfcells=len(uzf_pkdat),
        packagedata=uzf_pkdat,
        perioddata=uzf_spd,
        budget_filerecord=f"{gwfname}.uzf.bud",
        pname="UZF-1",
        filename=f"{gwfname}.uzf",
    )

    # Instantiating MODFLOW 6 output control package for flow model
    flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord="{}.hds".format(gwfname),
        budget_filerecord="{}.cbc".format(gwfname),
        headprintrecord=[
            ("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")
        ],
        saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )

    return sim

### Function for defining GWE models

Similarly, multiple GWE models will be defined.  Thus, this function can do the work for us multiple times.  This function will return the same GWT model setup each time it is called.

In [ ]:
def add_gwe_model(sim, gwename):

    # Instantiating MODFLOW 6 groundwater transport package
    gwe = flopy.mf6.MFModel(
        sim,
        model_type="gwe6",
        modelname=gwename,
        model_nam_file="{}.nam".format(gwename),
    )
    gwe.name_file.save_flows = True

    # Instantiate a solution
    imsgwe = flopy.mf6.ModflowIms(
        sim,
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="BICGSTAB",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename="{}.ims".format(gwename),
    )
    sim.register_ims_package(imsgwe, [gwe.name])

    # Instantiating MODFLOW 6 transport discretization package
    flopy.mf6.ModflowGwedis(
        gwe,
        length_units=length_units,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        pname="DIS",
        filename="{}.dis".format(gwename),
    )

    # Instantiating MODFLOW 6 transport initial concentrations
    flopy.mf6.ModflowGweic(
        gwe,
        strt=strt_conc,
        pname="IC",
        filename="{}.ic".format(gwename)
    )

    # Instantiating MODFLOW 6 transport advection package
    flopy.mf6.ModflowGweadv(
        gwe,
        scheme="TVD",
        pname="ADV",
        filename="{}.adv".format(gwename)
    )

    # Instantiating MODFLOW 6 transport dispersion package
    flopy.mf6.ModflowGwecnd(
        gwe,
        xt3d_off=False,
        alh=alphal,
        ath1=alphat,
        ktw=ktw,
        kts=lambda_s1,
        pname="CND",
        filename="{}.cnd".format(gwename),
    )

    # Instantiating MODFLOW 6 transport mass storage package (formerly "reaction" package in MT3DMS)
    flopy.mf6.ModflowGweest(
        gwe,
        porosity=n1,
        cps=cps,
        rhos=rhos,
        packagedata=[cpw, rhow, lhv],
        pname="EST",
        filename=f"{gwename}.est",
    )
    
    # Instantiating MODFLOW 6 energy transport source-sink mixing package
    uzepackagedata = [(iuz, T_az) for iuz in range(nlay)]
    uzeperioddata = {
        0: [[0, "INFILTRATION", "temp"]],
    }

    if "scen" in gwename:
        temp_ts_dict_entry = temp_ts_dict_2
    else:
        temp_ts_dict_entry = temp_ts_dict

    flopy.mf6.ModflowGweuze(
        gwe,
        flow_package_name="UZF-1",
        boundnames=False,
        save_flows=True,
        print_input=True,
        print_flows=True,
        print_temperature=True,
        timeseries=temp_ts_dict_entry,
        temperature_filerecord=gwename + ".uze.bin",
        budget_filerecord=gwename + ".uze.bud",
        packagedata=uzepackagedata,
        uzeperioddata=uzeperioddata,
        pname="UZE",
        filename=f"{gwename}.uze",
    )

    # Instantiating MODFLOW 6 transport constant concentration package
    flopy.mf6.ModflowGwectp(
        gwe,
        stress_period_data=ctp_spd,
        pname="CTP",
        filename="{}.ctp".format(gwename),
    )
    
    # Instantiating MODFLOW 6 transport source-sink mixing package
    flopy.mf6.ModflowGwessm(
        gwe,
        sources=[[]],
        pname="SSM",
        filename="{}.ssm".format(gwename)
    )

    # Instantiate MODFLOW 6 heat transport output control package
    flopy.mf6.ModflowGweoc(
        gwe,
        budget_filerecord="{}.cbc".format(gwename),
        temperature_filerecord="{}.ucn".format(gwename),
        temperatureprintrecord=[
            ("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")
        ],
        saverecord=[("TEMPERATURE", "LAST"), ("BUDGET", "LAST")],
        printrecord=[("TEMPERATURE", "LAST"), ("BUDGET", "LAST")],
        pname="OC",
    )

    return sim

### Create the simulation and add the models

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6", version="mf6"
)

# Instantiating MODFLOW 6 time discretization
flopy.mf6.ModflowTdis(
    sim, nper=nper, perioddata=per_mf6, time_units=time_units
)

# Add the first GWF model
gwfnamebase = 'gwf-base'
sim = add_gwf_model(sim, gwfnamebase, eps=3.6)

# Add corresponding GWE model
gwenamebase = 'gwe-base'
sim = add_gwe_model(sim, gwenamebase)

# Connect GWF1 to GWE1
flopy.mf6.ModflowGwfgwe(
    sim,
    exgtype="GWF6-GWE6",
    exgmnamea=gwfnamebase,
    exgmnameb=gwenamebase,
    filename="{}.gwfgwe".format(gwenamebase),
)

# Add the second GWF model
gwfnamescen = 'gwf-scen'
sim = add_gwf_model(sim, gwfnamescen, eps=7.2)

# Add corresponding GWE model
gwenamescen = 'gwe-scen'
sim = add_gwe_model(sim, gwenamescen)

# Connect GWF2 to GWE2
flopy.mf6.ModflowGwfgwe(
    sim,
    exgtype="GWF6-GWE6",
    exgmnamea=gwfnamescen,
    exgmnameb=gwenamescen,
    filename="{}.gwfgwe".format(gwenamescen),
)

sim.write_simulation(silent=False)

success, buff = sim.run_simulation(silent=False, report=True)
assert success, buff


### Load and animate UZF & UZE output

In [ ]:
# load temperature, model 1
gwe1 = sim.get_model(gwenamebase)
gwe2 = sim.get_model(gwenamescen)

temp_base = gwe1.output.temperature().get_alldata()
temp_scen = gwe2.output.temperature().get_alldata()

# Water content output
pth = sim.sim_path
flb = gwfnamebase + '.uzfobs'
uzdat_base = pd.read_csv(os.path.join(pth, flb))

fls = gwfnamescen + '.uzfobs'
uzdat_scen = pd.read_csv(os.path.join(pth, fls))

# Timing information for printing on animation
uzdat_base['time'] = uzdat_base['time'] / 86400
uzdat_base['time'] = uzdat_base['time'].astype(int)
uzdat_base.set_index('time', inplace=True)

uzdat_scen['time'] = uzdat_scen['time'] / 86400
uzdat_scen['time'] = uzdat_scen['time'].astype(int)
uzdat_scen.set_index('time', inplace=True)

# Mine all observations from obs names
depths_base = uzdat_base.columns
depths_base = [round(float(d.strip().split("=")[-1]), 2) for d in depths_base if "=" in d]

depths_scen = uzdat_scen.columns
depths_scen = [round(float(d.strip().split("=")[-1]), 2) for d in depths_scen if "=" in d]

norm = plt.Normalize(
    0.5, 20.0
)

layelvs = [top] + botm
cell_half_thk = [round((up - down) / 2, 2) for (up, down) in zip(layelvs[:-1], layelvs[1:])]
node_elvs = np.array(layelvs[:-1]) - np.array(cell_half_thk)
y = layelvs[0:20]
x = [0.5 for i in np.arange(len(y))]

points_uze = np.array([x, y]).T.reshape(-1, 1, 2)
points_scen = np.array([x, y]).T.reshape(-1, 1, 2)
segments_uze = np.concatenate([points_uze[:-1], points_uze[1:]], axis=1)
segments_scen = np.concatenate([points_scen[:-1], points_scen[1:]], axis=1)

temp_prof1 = temp_base[0, :len(y), 0, 0]
lc_uze = LineCollection(segments_uze, cmap="rainbow", norm=norm, array=temp_prof1, linewidth=50)

temp_prof2 = temp_scen[0, :len(y), 0, 0]
lc_scen = LineCollection(segments_scen, cmap="rainbow", norm=norm, array=temp_prof2, linewidth=50)


In [ ]:
# Animate results

matplotlib.rcParams['animation.embed_limit'] = 2**128

fig, (ax1, ax2, ax3) = plt.subplots(
    nrows=1,
    ncols=3,
    figsize=(7.5, 4),
    sharey=True,
    gridspec_kw=dict(width_ratios=[3, 1, 1], wspace=0.2)
)

ax1.set_ylabel("Depth (m)")
ax1.set_xlabel("Water Content")
(l1,) = ax1.plot([], [], "b-", mfc="none", label=r"$\theta$")
(l2,) = ax1.plot([], [], "r-", mfc="none", label=r"$\omega$")
line = [l1, l2]
#scen_line = [l2]  # added 5/30 - but wrong I think
ax1.legend(loc="lower right")
ax1.set_xlim(0.0, 0.25)
ax1.set_ylim(54.0, 60.1)
# ax1.set_yticks(layelvs[:len(y)], minor=True)
# ax1.yaxis.grid(True, which='minor')
for yline in layelvs[:len(y)]:
    ax1.axhline(yline, color='gray', linewidth=0.5)

times = uzdat_base.index

# UZE temps
ax2.set_title("UZF/UZE")
ax2.set_ylabel("")
ax2.set_xlabel("")
ax2.set_xlim(0.4, 0.6)
ax2.set_ylim(54.0, 60.1)
ax2.add_collection(lc_uze)
ax2.get_xaxis().set_visible(False)
ax2.yaxis.set_ticks_position('both')
ax2.set_yticks(layelvs[0:20], minor=True)
# ax2.tick_params(axis='y', which='both', labelleft='off', labelright='off')

ax3.set_title("SCEN")
ax3.set_ylabel("")
ax3.set_xlabel("")
ax3.set_xlim(0.4, 0.6)
ax3.set_ylim(54.0, 60.1)
ax3.add_collection(lc_scen)
ax3.get_xaxis().set_visible(False)
# ax3.yaxis.tick_left(False)
ax3.yaxis.tick_right()
ax3.yaxis.set_ticks_position('both')
# ax3.yaxis.set_label_position("right")
ax3.tick_params(axis='y', which='both', labelleft=False, labelright=True)

def init():
    ax1.set_title("Time = %.2f years" % (times[0]))

def update(j):
    # Moisture Content
    XBase = uzdat_base.iloc[j].values
    XScen = uzdat_scen.iloc[j].values
    t = times[j] / 365
    line[0].set_data(XBase, depths_base)
    line[1].set_data(XScen, depths_scen)
    ax1.set_title("Time = %.2f years" % (t))

    # UZE temperature pulse
    temp_prof1 = temp_base[j, :len(y), 0, 0]
    lc_uze.set_array(temp_prof1)

    # NWT temperature pulse
    temp_prof2 = temp_scen[j, :len(y), 0, 0]
    lc_scen.set_array(temp_prof2)

    return

ani = animation.FuncAnimation(fig, update, interval=100, frames=365)  # frames=len(times)
HTML(ani.to_jshtml())